In [ ]:

# -*- coding: utf-8 -*-
from oandapyV20 import API
from oandapyV20.exceptions import V20Error
from oandapyV20.endpoints.pricing import PricingStream
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.instruments as instruments

import json
import datetime
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta

import time
import random
import csv

get_ipython().run_line_magic('matplotlib', 'inline')
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
import pickle
import math
import glob
import talib as ta
import shutil
import os

import schedule
from tqdm.notebook import tqdm
from sklearn.model_selection import GridSearchCV
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score
from pprint import pprint

#自分のアカウント、トークンをセット
accountID = "101-009-16415310-001"
access_token = '860c2d2dbbcd01b6d95939b6af6b5981-8664724ce0870220c55161d704137df6'

api = API(access_token=access_token, environment="practice")
        
# Oandaからcandleデータを取得する。
def getCandleDataFromOanda(instrument, api, date_from, date_to, granularity):
    params = {
        "from": date_from.isoformat(),
        "to": date_to.isoformat(),
        "granularity": granularity,
    }
    r = instruments.InstrumentsCandles(instrument=instrument, params=params)
    return api.request(r)

def oandaJsonToPythonList(JSONRes):
    
    data = []
    for res in JSONRes['candles']:
        temp = [((datetime.datetime.fromisoformat(res['time'][:19]))+datetime.timedelta(hours=+9)).strftime('%Y-%m-%d %H:%M')]#indexとなる時間を追加
        temp.extend([res['volume'],
            res['mid']['o'],
            res['mid']['h'],
            res['mid']['l'],
            res['mid']['c'],
            ])
        data.append(temp)
    return data

def logarism(x):
    x = abs(x)
    return math.log(x)


def get_now_data(time, period_hours, money, foot, back_hours = 0):
    
    all_data = []
    #time = time.replace(hour=7, minute=0, second=0, microsecond=0)
    #入力された日の当日の7時
    
    global PAST
    past = datetime.timedelta(hours=-PAST)
    
    NY = datetime.timedelta(hours=-14)
    back = datetime.timedelta(hours=-back_hours)
        
    date_to = time + NY + datetime.timedelta(seconds=-1) + back + past

    date_from = date_to + datetime.timedelta(hours=-period_hours)
    
    ret = getCandleDataFromOanda(money, api, date_from, date_to, foot)
    month_data = oandaJsonToPythonList(ret)#取得したデータを格納

    all_data.extend(month_data)#データを追加

    # pandas DataFrameへ変換
    df = pd.DataFrame(all_data)

#     print(date_from, date_to)
#     print(df)
    
    df.columns = ['Datetime','Volume', 'Open', 'High', 'Low', 'Close']
    
    df['Volume'] = df['Volume'].astype('double')
    df['Open'] = df['Open'].astype('double')
    df['Close'] = df['Close'].astype('double')
    df['High'] = df['High'].astype('double')
    df['Low'] = df['Low'].astype('double')
    
    
    close = np.array(df["Close"]).astype(float)
#     #以下、talibを用いてテクニカル指標（今回の学習で用いる特徴量）を算出しdf_feature入れる
#     #単純移動平均は、単純移動平均値とその日の終値の比を特徴量として用いる
    df["SMA2"]= ta.SMA(close, timeperiod=2) / close
#     df["SMA3"]= ta.SMA(close, timeperiod=3) / close
#     df["SMA4"]= ta.SMA(close, timeperiod=4) / close
#     df["SMA5"]= ta.SMA(close, timeperiod=5) / close
#     df["SMA6"]= ta.SMA(close, timeperiod=6) / close
#     df["SMA7"]= ta.SMA(close, timeperiod=7) / close
#     df["SMA8"]= ta.SMA(close, timeperiod=8) / close
#     df["SMA9"]= ta.SMA(close, timeperiod=9) / close
#     df["SMA10"]= ta.SMA(close, timeperiod=10) / close
#     df["RSI"] = ta.RSI(close, timeperiod=9) / close
    #ボリンジャーバンド Main 
    upper, middle, lower = ta.BBANDS(close, timeperiod=10, nbdevup=2, nbdevdn=2)
    df["BBANDS+σ"] = upper / close * 1000
    df["BBANDS-σ"] = lower / close * 1000
    
    #df['Rate']=(df['Close']-df['Open'])/df['Open']
    
#     df['Open'] = df['Open'].apply(lambda x: logarism(x))
#     df['Close'] = df['Close'].apply(lambda x: logarism(x))
    
    df["SMA2"]= df["SMA2"].apply(lambda x: logarism(x)) * 1000
#     df["SMA3"]= df["SMA3"].apply(lambda x: logarism(x)) * 1000
#     df["SMA4"]= df["SMA4"].apply(lambda x: logarism(x)) * 1000
#     df["SMA5"]= df["SMA5"].apply(lambda x: logarism(x)) * 1000
#     df["SMA6"]= df["SMA6"].apply(lambda x: logarism(x)) * 1000
#     df["SMA7"]= df["SMA7"].apply(lambda x: logarism(x)) * 1000
#     df["SMA8"]= df["SMA8"].apply(lambda x: logarism(x)) * 1000
#     df["SMA9"]= df["SMA9"].apply(lambda x: logarism(x)) * 1000
#     df["SMA10"]= df["SMA10"].apply(lambda x: logarism(x)) * 1000
    
    
    df['Rate']=(df['Close'].astype('double')-df['Open'].astype('double'))/df['Open'].astype('double')
    
    df['Rate'] = df['Rate']*1000
    
    df = df.dropna()
    
#     df['Open'] = df['Open']/df['Close']
    
#     df = df.drop('Open', axis=1)
#     df = df.drop('High', axis=1)
#     df = df.drop('Low', axis=1)
#     df = df.drop('Close', axis=1)
#     df = df.drop('Volume', axis=1)
    
    df = df.set_index('Datetime')
    
    
    return df



def get_csv(back_hours, hours, money):
    
    time = datetime.datetime.now() 
    
    df = get_now_data(time, period_hours=hours, money=money, foot = FOOT , back_hours=back_hours)
    date_to = (time+datetime.timedelta(hours=-back_hours)+datetime.timedelta(seconds=-1)).strftime("%Y-%m-%d")
    date_from = (time + datetime.timedelta(hours=-hours)+datetime.timedelta(hours=-back_hours)).strftime("%Y-%m-%d")
    df.to_csv("csv/"+str(money)+"_"+ date_from + "_" + date_to +"_"+str(hours)+'.csv', mode = 'w')
    print('"'+str(money)+"_"+ date_from + "_" + date_to +"_"+str(hours)+'.csv', end='", ')
    
    return df

def get_moneys_csv(back_hours, hours, moneylist):
    csvlist = []
    
    time = datetime.datetime.now()
    date_to = (time+datetime.timedelta(hours=-back_hours)+datetime.timedelta(seconds=-1)).strftime("%Y-%m-%d")
    date_from = (time + datetime.timedelta(hours=-hours)+datetime.timedelta(hours=-back_hours)).strftime("%Y-%m-%d")
    
    for i in range (len(moneylist)):
        get_csv(back_hours, hours, money=moneylist[i])
        csvlist.extend([str(moneylist[i])+"_"+ date_from + "_" + date_to +"_"+str(hours)+'.csv'])
    
    return csvlist
        
def classify_two(x, c):
        if x<0:
            c[0]+=1
            return -1
        elif 0<=x:
            c[1]+=1
            return 1
        
def classify_random(x):
        ram = random.randint(0,1)
        if ram==0:
            return -1
        elif ram==1:
            return 1
        
        
def predict(dataname, moneyname, test_hours, back_hour, datanum):

    df = pd.read_csv("csv/"+dataname, index_col='Datetime')
    
    count = [0,0]
    #df['class'] = (df["Rate"]).apply(lambda x: classify(x, count, 0.001))
    df['class'] = (df["Rate"]).apply(lambda x: classify_two(x, count))
    df['class'] = df['class'].shift(-1)
    print("Train Data")
    print(" LOW/HIGH")
    print(count)
    
    X = df.drop('class', axis=1)
    
    
    y = df['class']
    X = X.drop(X.index[len(X)-1])
    y = y.dropna()
    

    df_now = get_now_data(datetime.datetime.now(), period_hours=test_hours, money=moneyname, foot = FOOT, back_hours = back_hour)
    print("money: "+moneyname)
    
    count = [0,0]
    #df['class'] = (df["Rate"]).apply(lambda x: classify(x, count, 0.001))
    df_now['class'] = (df_now["Rate"]).apply(lambda x: classify_two(x, count))
    df_now['class'] = df_now['class'].shift(-1)
    print("NOW Data from" +str(test_hours)+ "hours ago")
    print(" LOW/MID/HIGH")
    print(count)
    
    print("learning data")
    print(df.head(1).index.values, end=' to ')
    print(df.tail(1).index.values)
    
    print("")
    print("test data")
    print(df_now.head(1).index.values,end=' to ')
    print(df_now.tail(1).index.values)
    print("")
    X_now = df_now.drop('class', axis=1)
    
    X_next = X_now.tail(1) #最終予測に利用
    y_now = df_now['class']
    X_now = X_now.drop(X_now.index[len(X_now)-1])
    y_now = y_now.dropna()
    
    X_train = X
    y_train = y
    X_test = X_now
    y_test = y_now
    
    
#     sns.pairplot(df)
    
    
    names = [
        "Nearest Neighbors",
#         "Linear SVM",
#         "RBF SVM",
#         "Decision Tree",
#         "Random Forest",
#         "AdaBoost",
#         "Naive Bayes",
#         "Linear Discriminant Analysis",
#         "Quadratic Discriminant Analysis"
    ]
    classifiers = [
    KNeighborsClassifier(3),
#     SVC(kernel="linear", C=0.025),
#     SVC(gamma=2, C=1),
#     DecisionTreeClassifier()#(max_depth=10),
#     RandomForestClassifier(),#max_depth=10, n_estimators=10, max_features=1
#     AdaBoostClassifier(),
#     GaussianNB(),
#     LinearDiscriminantAnalysis(),
#     QuadraticDiscriminantAnalysis()
    ]
    
    accuracy_scores = []
    precision_scores = []
    next_class = []
    profit = []
    
    clearlist = []
    
    for name, clf in zip(names, classifiers):
        print(name)
        
#         def randomforest_cv(n_estimators, min_samples_split, max_features, max_depth):
#             val = cross_val_score(
#                 RandomForestClassifier(
#                     n_estimators=int(n_estimators),
#                     min_samples_split=int(min_samples_split),
#                     max_features=max_features,
#                     random_state=0,
#                     max_depth = max_depth,
#                     #max_leaf_nodes = max_leaf_nodes
#                 ),
#                 X_train, y_train,
#                 scoring = 'accuracy',
#                 cv = 3, # 3-fold
#                 n_jobs = -1 # use all CPUs
#             ).mean()
#             return val

#         randomforest_cv_bo = BayesianOptimization(
#             randomforest_cv,
#             {'n_estimators': (10, 250),
#             'min_samples_split': (2, 25),
#             'max_features': (0.1, 0.999),
#             "max_depth": (1, 100),
#             #"max_leaf_nodes": (1,100),
#             }
#         )

#         gp_params = {"alpha": 1e-5}
#         randomforest_cv_bo.maximize(n_iter=50, **gp_params)
#         print(randomforest_cv_bo.max)
    
        
#         clf = RandomForestClassifier(
#                     n_estimators=int(randomforest_cv_bo.max['params']['n_estimators']),
#                     min_samples_split=int(randomforest_cv_bo.max['params']['min_samples_split']),
#                     max_features=randomforest_cv_bo.max['params']['max_features'],
#                     max_depth = randomforest_cv_bo.max['params']['max_depth'],
#                     random_state=0,
#                     #max_leaf_nodes = randomforest_cv_bo.max['params']['max_leaf_nodes']
#                 )

        clf.fit(X_train, y_train)
        result = clf.predict(X_test)
        profit, values = calc_profit(result, y_test)
        precision = precision_score(y_test, result, average=None)
        accuracy = accuracy_score(y_test, result)
        confusion = confusion_matrix(y_test, result)
        
        #print(df)
#         print(clf.feature_importances_)
        
        if(1):
#         if(precision[0]!=1 and precision[1]!=1 and abs(confusion[0][0]-confusion[1][1])<(confusion[0][0]+confusion[1][1])/4):
#         if(precision[0]>0.4 and precision[1]>0.4):
#             score = clf.score(X_test, y_test)
            next_class.extend(clf.predict(X_next))

            print('accuracy_score: ', end='')
            print(accuracy_score(y_test, result))
            accuracy_scores.extend([accuracy_score(y_test, result)])
            print('confusion_matrix: ')
            print(confusion_matrix(y_test, result))
            print('precision_score: ', end='')
            print(precision_score(y_test, result, average=None))
            precision_scores.extend([precision_score(y_test, result, average=None)])

            print(str(len(result))+"hours trade")
            print("profit: "+str(profit))

            print('', flush=True)

            model_name = moneyname + "_" + name + "_" + str(datanum) +"_"+str(accuracy)+".sav"
            #clearlist.extend([model_name])
            print("CLEAR: " + model_name)
            pickle.dump(clf, open("models/"+model_name, 'wb'))

    #print(clearlist)
    return accuracy_scores, precision_scores, next_class, profit

def make_models(test_hours, namelist, moneylist, back_hours, datanum):
    for i in range(len(namelist)):
        print(namelist[i], flush=True)
        acc, pre, next_class, profit = predict(namelist[i], moneylist[i], test_hours, back_hours, datanum)
        print(acc)


def calc_profit(predict_result, true_data):
    win=0
    lose=0
    for i in range(len(predict_result)):
            if (predict_result[i]==-1):#Low予想のとき
                if (predict_result[i]==true_data[i]): #予測成功
                    win+=1
                else:
                    lose+=1
            elif(predict_result[i]==1):#High予想の時
                if (predict_result[i]==true_data[i]): #予測成功
                    win+=1
                else:
                    lose+=1
    profit = win*0.9-lose #利率を算出
    values = predict_result*profit #予測結果×利率により予測の価値を算出
    return profit, values

def plot_result(index, predict_result, true_data):
    plt.figure(figsize=(20,5))
    index = index.astype(float)

    color = []
    marker = []
    accuracy = []
    
    for x in range(0,len(predict_result)):
        if true_data.values[x] == predict_result[x]:
            color.extend(['blue'])
            accuracy.extend([1])
        
        elif predict_result[x]==0:
            color.extend(['green'])
            accuracy.extend([0])
        else:
            color.extend(['red'])
            accuracy.extend([-1])


    plt.scatter(index.index.values, index["Rate"].values, color=color)

    plt.xticks(rotation=90)
    plt.show()
    
    return accuracy


def ensemble(model_names, moneyname, hours):
    

    global PAGE
    
    temp = PAGE
#     PAGE = trader.select_money(moneyname, temp)
    
    df_now = get_now_data(datetime.datetime.now()+datetime.timedelta(hours=-ENSEM_BACK), period_hours=hours, money=moneyname, foot = FOOT, back_hours = NIGHT)###########
    print("money: "+moneyname)
    count = [0,0]
    
    df_now['class'] = (df_now["Rate"]).apply(lambda x: classify_two(x, count))
    df_now['class'] = df_now['class'].shift(-1)
    print("NOW Data from" +str(hours)+ "hours ago")
    print(" LOW/HIGH")
    print(count)

    print("")
    print("test data")
    print(df_now.head(1).index.values,end=' to ')
    print(df_now.tail(1).index.values)
    print("")
    X_now = df_now.drop('class', axis=1)
    
    X_next = X_now.tail(1) #最終予測に利用
    y_now = df_now['class']
    X_now = X_now.drop(X_now.index[len(X_now)-1])
    y_now = y_now.dropna()
    
    values = []
    values_sum = []
    win=0
    lose=0
    answer = 0
    
#     #add DAY/2 前半でprecisionを作成、後半で実際の的中率を検討 ver3.1
#     size = len(X_now)
#     X_test = X_now.tail(int(size/2)-1)
#     X_now = X_now.head(int(size/2))
    
#     y_test = y_now.tail(int(size/2)-1)
#     y_now = y_now.head(int(size/2))
    #print(len(X_test), len(y_test), len(X_now), len(y_now))
    X_test = X_now
    y_test = y_now
    
    
    for x in range(len(model_names)):
        clf = pickle.load(open(model_names[x], 'rb')) #clfをload
        print(model_names[x], flush=True)
        result = clf.predict(X_now) #predict
        precision = precision_score(y_now, result, average=None)
        #if(precision[0]!=0 and precision[1]!=0 and precision[0]!=1 and precision[1]!=1):
        if(1):
       
            #score表示
            print("accuracy_score: ", end="")
            accuracy = accuracy_score(y_now, result)
            print(accuracy)
            print("precision_score: ", end="")
            print(precision)
            print(confusion_matrix(y_now, result))

            profit, values = calc_profit(result, y_now) #予測結果について利率と、予測*利率による予測の信頼性を算出
            
            
            #answer += clf.predict(X_next)[0]*profit
            answer += clf.predict(X_next)[0]*accuracy_score(y_now, result)
            
            result = clf.predict(X_test)
             #predict
            
            global RANDOM
            if(RANDOM==1):
                result = np.vectorize(classify_random)(result)
            
            ####valuesの算出方法で予測精度が決まる
            
            #values = result*accuracy #profit:-16
            
            #values = np.where(result<0, result*precision[0], result*precision[1]) #profit:-14
            values = result  #profit:-18
            
            
#             R=0.3  #R=0.37で-13.0,  R=0.35で-16 R=0.45で15, R=0.5で-15
        
#             if(precision[0]<R):
#                 precision[0] = 0
#             if(precision[1]<R):
#                 precision[1] = 0
#             values = np.where(result<0, result*precision[0], result*precision[1])

            
            #values = np.where(result<0, result*(((precision[0])*100)**2), (result*(((precision[1])*100)**2)))
            
            if len(values_sum)==0:#1回目は配列をコピー
                values_sum = values
            else:
                values_sum += values #２回目からは加算
            print(values)
    
        #plot_result(X_now, result, y_now)
    if(answer>0):
        print("HIGH")
        hl = "HIGH"
#         trader.high_entry()
        
    else:
        print("LOW")
        hl = "LOW"
#         trader.low_entry()    
    print(values_sum)
    
    if(values_sum==[]):
        print("No model")
        return 0
    
    ensemble_result = np.where(values_sum<0, -1, 1) #クラス分類
    
    print("")
    print("ensemble_result: ",end='')
    print(accuracy_score(y_test.values, ensemble_result))
    print(ensemble_result)
    print(y_test.values)
    print(values_sum)
    print("precision_score: ", end="")
    print(precision_score(y_test, ensemble_result, average=None))
    print('confusion_matrix: ')
    print(confusion_matrix(y_test, ensemble_result))
    print("profit: ",end='')
    profit, values = calc_profit(ensemble_result, y_test)
    print(profit)
    print("next: ",end='', flush=True)
    print(answer)
    
#     print(X_next.head(1))
#     accuracy = plot_result(X_test, ensemble_result, y_test)
    
    global profit_sum
    profit_sum += profit
    
    global RESULT
    RESULT.extend([moneyname+": "+hl])
    
    global TEST
    TEST.extend([moneyname])
    TEST.extend([accuracy_score(y_test.values, ensemble_result)])
    
    return accuracy
        
    
def compere_ensemble(models_array, moneys_array, hours):
    results = []
    
    for i in range(len(moneys_array)):
        results.append(np.array(ensemble(models_array[i], moneys_array[i], hours)).astype('int'))
    
    np.savetxt('out.csv',results,delimiter=',')
    print(results)
    
def ensemble_set(models_array, moneys_array, hours):
    
    for i in range(len(models_array)):
        print(ensemble(models_array[i], moneys_array[i], hours))    

def plot_rates(hours, moneys):
    
    result = []
    accord = []
    plt.figure(figsize=(40,20))
    for i in range(len(moneys)):
#         df_now = get_now_data(datetime.datetime.now(), period_hours=hours, money=moneys[i], foot = 'D', back_hours = 0)
        df_now = get_now_data(datetime.datetime.now(), period_hours=hours, money=moneys[i], foot = FOOT, back_hours = 0)
        plt.plot(df_now.index.values, df_now["Rate"].values)
        count = [0,0]
        df_now['class'] = (df_now["Rate"]).apply(lambda x: classify_two(x, count))
        result.append(df_now["class"].values)
    
    win = 0
    lose = 0
    
    for l in range(len(result[0])):
        temp = 0
        for m in range(len(moneys)):
            if result[m][l]==1:
                temp+=1
        if(temp==len(moneys)/2):
            accord.extend([-1])
            lose += len(moneys)/2
            win += len(moneys)/2
        else:
            accord.extend([temp])
            win += max(temp, len(moneys)-temp)
            lose += min(temp, len(moneys)-temp)
    
    
    plt.xticks(rotation=90)
    plt.hlines([0], 0, hours, "blue", linestyles='dashed')     # hlines
    plt.show()
    print(accord)
    
    pay = 1000
    pay_sum = pay*len(moneys)
    
    print("win: ", win)
    print("lose: ", lose)
    print("rate: ", win/(win+lose))
    print("profit: ", (win*0.9-lose)*pay_sum)

COUNT = 0
PAGE = 0
    
def job():
    global COUNT
    
    time.sleep(5)
    print("START")
    
    if COUNT == 0:
        print("Set up browser...")
        trader.setup_demo()
        time.sleep(3)
    else:
        print("Ready for entry...")
        time.sleep(13)
    main()
    COUNT += 1
    
    #print(RESULT)
    print('')
    print("profit_sum: ", end='')
    print(profit_sum)
    RESULT = []
    print("Rebuild models...")
    get_csv_and_make_models()
    print('')
    print('finished: ', end='')
    print(datetime.datetime.now())
    print('waiting...', flush=True)


    
    
def main():
    
#     models = [glob.glob('models/USD_JPY*.sav'), glob.glob('models/EUR_JPY*.sav'), glob.glob('models/AUD_JPY*.sav'),
#               glob.glob('models/GBP_JPY*.sav'), glob.glob('models/NZD_JPY*.sav'), glob.glob('models/CAD_JPY*.sav'),
#               glob.glob('models/CHF_JPY*.sav')]#, glob.glob('ZAR_JPY*.sav')]

#     models = [glob.glob('models/*.sav'), glob.glob('models/*.sav'), glob.glob('models/*.sav'),
#               glob.glob('models/*.sav'), glob.glob('models/*.sav'), glob.glob('models/*.sav'),
#               glob.glob('models/*.sav')]
    
#     models = [glob.glob('models/*.sav'), glob.glob('models/*.sav'), 
#               glob.glob('models/*.sav'), glob.glob('models/*.sav'), 
#               glob.glob('models/*.sav')]
    
#     moneys = ['USD_JPY', 'EUR_JPY', 'AUD_JPY', 'GBP_JPY', 'NZD_JPY', 'CAD_JPY', 'CHF_JPY']#, 'ZAR_JPY']
#     moneys = ['USD_JPY', 'EUR_USD', 'EUR_JPY', 'GBP_JPY', 'AUD_JPY']

#     models = [
# #         glob.glob('models/AUD_JPY*.sav'),
#         glob.glob('models/AUD_NZD*.sav'),
#         glob.glob('models/AUD_USD*.sav'),
#         glob.glob('models/CAD_JPY*.sav'),
#         glob.glob('models/CHF_JPY*.sav'),
# #         glob.glob('models/EUR_AUD*.sav'),
# #         glob.glob('models/EUR_GBP*.sav'),
#         glob.glob('models/EUR_JPY*.sav'),
# #         glob.glob('models/EUR_USD*.sav'),
# #         glob.glob('models/GBP_AUD*.sav'),
# #         glob.glob('models/GBP_JPY*.sav'),
#         glob.glob('models/GBP_USD*.sav'), 
# #         glob.glob('models/NZD_JPY*.sav'),
#         glob.glob('models/NZD_USD*.sav'),
# #         glob.glob('models/USD_CAD*.sav'),
#         glob.glob('models/USD_CHF*.sav'),
#         glob.glob('models/USD_JPY*.sav')
#     ]

#     models = [
#             glob.glob('models/AUD_JPY*.sav'),
# #             glob.glob('models/AUD_NZD*.sav'),
# #             glob.glob('models/AUD_USD*.sav'),
# #             glob.glob('models/CAD_JPY*.sav'),
# #             glob.glob('models/CHF_JPY*.sav'),
# #             glob.glob('models/EUR_AUD*.sav'),
# #             glob.glob('models/EUR_GBP*.sav'),
#             glob.glob('models/EUR_JPY*.sav'),
#             glob.glob('models/EUR_USD*.sav'),
# #             glob.glob('models/GBP_AUD*.sav'),
#             glob.glob('models/GBP_JPY*.sav'),
# #             glob.glob('models/GBP_USD*.sav'), 
# #             glob.glob('models/NZD_JPY*.sav'),
# #             glob.glob('models/NZD_USD*.sav'),
# #             glob.glob('models/USD_CAD*.sav'),
# #             glob.glob('models/USD_CHF*.sav'),
#             glob.glob('models/USD_JPY*.sav')
#         ]
    
#     models = [glob.glob('models/*.sav'), glob.glob('models/*.sav'), glob.glob('models/*.sav'),
#           glob.glob('models/*.sav'), glob.glob('models/*.sav'), glob.glob('models/*.sav'),
#           glob.glob('models/*.sav'), glob.glob('models/*.sav'), glob.glob('models/*.sav'),
#           glob.glob('models/*.sav'), glob.glob('models/*.sav'), glob.glob('models/*.sav'), 
#           glob.glob('models/*.sav'),
#           glob.glob('models/*.sav'), glob.glob('models/*.sav'), glob.glob('models/*.sav'),
#           glob.glob('models/*.sav')]

#     moneys = [
# #         'AUD_JPY',
#         'AUD_NZD',
#         'AUD_USD',
#         'CAD_JPY',
#         'CHF_JPY',
# #         'EUR_AUD',
# #         'EUR_GBP',
#         'EUR_JPY',
# #         'EUR_USD',
# #         'GBP_AUD',
# #         'GBP_JPY',
#         'GBP_USD',
# #         'NZD_JPY',
#         'NZD_USD',
# #         'USD_CAD',
#         'USD_CHF',
#         'USD_JPY'
#     ]

#     moneys = [
#             'AUD_JPY',
#             'AUD_NZD',
#             'AUD_USD',
#             'CAD_JPY',
#             'CHF_JPY',
#             'EUR_AUD',
#             'EUR_GBP',
#             'EUR_JPY',
#             'EUR_USD',
#             'GBP_AUD',
#             'GBP_JPY',
#             'GBP_USD',
#             'NZD_JPY',
#             'NZD_USD',
#             'USD_CAD',
#             'USD_CHF',
#             'USD_JPY'
#         ]

#     moneys = [#page1
#             'AUD_JPY',
#     #         'AUD_NZD',
#     #         'AUD_USD',
#     #         'CAD_JPY',
#     #         'CHF_JPY',
#     #         'EUR_AUD',
#     #         'EUR_GBP',
#             'EUR_JPY',
#             'EUR_USD',
#     #         'GBP_AUD',
#             'GBP_JPY',
#     #         'GBP_USD',
#     #         'NZD_JPY',
#     #         'NZD_USD',
#     #         'USD_CAD',
#     #         'USD_CHF',
#             'USD_JPY'
#         ]


#     models = [glob.glob('models/GBP_JPY*.sav'), glob.glob('models/NZD_JPY*.sav'), glob.glob('models/NZD_USD*.sav')]
#     moneys = ['GBP_JPY', 'NZD_JPY', 'NZD_USD']
    
#     models = [glob.glob('models/*.sav'), glob.glob('models/*.sav'), glob.glob('models/*.sav'),
#               glob.glob('models/*.sav'), glob.glob('models/*.sav'), glob.glob('models/*.sav'),
#               glob.glob('models/*.sav')]
    
#     moneys = ['AUD_JPY', 'AUD_NZD', 'CAD_JPY', 'EUR_JPY', 'GBP_AUD', 'GBP_JPY', 'GBP_USD']
    
    global MAIN_MONEY
    moneys = MAIN_MONEY
    models = [glob.glob('models/*.sav')]

#     moneys = [
#               'AUD_USD',
#               'EUR_USD',
#               'USD_JPY',
#               'AUD_USD',
#               'AUD_NZD',
#               'GBP_USD',
#               'GBP_JPY',
#               'CAD_JPY',
#               'EUR_GBP'
#              ]
#     models = [
#             glob.glob('models/AUD_USD*.sav'),
#               glob.glob('models/CAD_JPY*.sav'),
#               glob.glob('models/CHF_JPY*.sav'),
#               glob.glob('models/EUR_JPY*.sav'),
#               glob.glob('models/GBP_JPY*.sav'),
#               glob.glob('models/GBP_JPY*.sav'),
#               glob.glob('models/GBP_JPY*.sav'),
#               glob.glob('models/GBP_USD*.sav'),
#               glob.glob('models/NZD_JPY*.sav')
#              ]
    
    #ensemble(CHF_models, "CHF_JPY", 60)   #選択した通貨に対してアンサンブル
    #compere_ensemble(models, moneys, 30)   #それぞれの通貨に対してアンサンブルした結果をcsvに返す
    #plot_rates(30,moneys)
    
    ensemble_set(models, moneys, hours)  #指定したmoneyで結果を出す
    print(RESULT)

    print('')
    print("profit_sum: ", end='')
    print(profit_sum)
    
def get_csv_and_make_models():
    
    shutil.rmtree('csv')
    shutil.rmtree('models')
    
    os.mkdir('csv')
    os.mkdir('models')
    
    global MLIST
    moneylist = MLIST
    
#     moneylist = [
#                 'AUD_USD',
#                  'CAD_JPY',
#                  'CHF_JPY',
#                  'EUR_JPY',
#                  'GBP_JPY',
#                  'GBP_JPY',
#                  'GBP_JPY',
#                  'GBP_USD',
#                  'NZD_JPY'
#                 ]
    
#     moneylist = ['USD_JPY', 'EUR_JPY', 'AUD_JPY', 'GBP_JPY', 'NZD_JPY', 'CAD_JPY', 'CHF_JPY']#, 'ZAR_JPY']
#     moneylist = ['CAD_JPY']

#     moneylist = [
#         'AUD_JPY',#+400
#         'AUD_NZD',#-18599
#         'AUD_USD', #-9000
#         'CAD_JPY',#-1500
#         'CHF_JPY',#-20000
#         'EUR_AUD',#-1499
#         'EUR_GBP',#19400
#         'EUR_JPY',
#         'EUR_USD',
#         'GBP_AUD',
#         'GBP_JPY',
#         'GBP_USD',
#         'NZD_JPY',
#         'NZD_USD',
#         'USD_CAD',
#         'USD_CHF',
#         'USD_JPY'
#     ]

#     moneylist = [#page1
#         'AUD_JPY',
# #         'AUD_NZD',
# #         'AUD_USD',
# #         'CAD_JPY',
# #         'CHF_JPY',
# #         'EUR_AUD',
# #         'EUR_GBP',
#         'EUR_JPY',
#         'EUR_USD',
# #         'GBP_AUD',
#         'GBP_JPY',
# #         'GBP_USD',
# #         'NZD_JPY',
# #         'NZD_USD',
# #         'USD_CAD',
# #         'USD_CHF',
#         'USD_JPY'
#     ]

#     moneylist = ['GBP_JPY', 'NZD_JPY', 'NZD_USD']
#     moneylist = ['USD_JPY']
    
    
#     array100 = get_moneys_csv(back_hours=5, hours=100, moneylist = moneylist)
#     array500 = get_moneys_csv(back_hours=BACK*2, hours=500, moneylist = moneylist) # 指定期間のcsvをsave
#     array1000 = get_moneys_csv(back_hours=BACK*2, hours=1000, moneylist = moneylist)
#     array1500 = get_moneys_csv(back_hours=hours, hours=1500, moneylist = moneylist)
#     array2000 = get_moneys_csv(back_hours=BACK*2, hours=2000, moneylist = moneylist)
#     array2500 = get_moneys_csv(back_hours=BACK*2, hours=2500, moneylist = moneylist)
    array4000 = get_moneys_csv(back_hours=BACK*2, hours=4000, moneylist = moneylist)
    
   #make_models(test_hours, namelist, moneylist, back_hours, datanum):
    
    #make_models(24, array100, moneylist, 1, 100)
#     make_models((hours), array500, moneylist, BACK, 500)
#     make_models((hours), array1000, moneylist, BACK, 1000)
#     make_models((hours), array1500, moneylist, 0, 1500)
#     make_models((hours), array2000, moneylist, BACK, 2000)
#     make_models((hours), array2500, moneylist, BACK, 2500)
    make_models((hours), array4000, moneylist, BACK, 4000)

NIGHT = 0
FOOT = 'H1'

hours = 24*5
hours += 20

BACK = hours #hours
ENSEM_BACK = BACK

PAST = 0
RANDOM = 0


#get_csv_and_make_models()
RESULT = []

profit_sum = 0

TEST = []
PROFIT = []
SUM  = 0
def test(num):
    global profit_sum, ENSEM_BACK, BACK, TEST, RESULT, PROFIT, SUM
    for i in range(num):
        get_csv_and_make_models()
        profit_sum = 0
        RESULT = []
        TEST.extend([BACK])
        main()
        PROFIT.extend([num,profit_sum])
        SUM +=profit_sum
        BACK += hours
        ENSEM_BACK = BACK

MLIST = []
MAIN_MONEY = []
def full_test():
    global MLIST, MAIN_MONEY, SUM, ENSEM_BACK, BACK
    moneylist = [
        'AUD_JPY',
        'AUD_NZD',
        'AUD_USD',
        'CAD_JPY',
        'CHF_JPY',
        'EUR_AUD',
        'EUR_GBP',
        'EUR_JPY',
        'EUR_USD',
        'GBP_AUD',
        'GBP_JPY',
        'GBP_USD',
        'NZD_JPY',
        'NZD_USD',
        'USD_CAD',
        'USD_CHF',
        'USD_JPY'
    ]
    
    for i in range(len(moneylist)):
        for j in range(len(moneylist)):
            MLIST.extend([moneylist[i]])
            MAIN_MONEY.extend([moneylist[j]])
            test(20)
            with open('result.csv', 'a') as f:
                writer = csv.writer(f)
                writer.writerow([MLIST[0], MAIN_MONEY[0], SUM])
#             print("BASE: " + MLIST[0], end=" ")
#             print("MONEY: " + MAIN_MONEY[0], end=" ")
#             print(SUM)
            SUM = 0
            hours = 100
            hours += 20
            BACK = hours #hours
            ENSEM_BACK = BACK
            MLIST = []
            MAIN_MONEY = []
            
    
full_test()
        
# get_csv_and_make_models()
# main()

# import trader
# trader.setup_demo()
# test(5)
# print(TEST)
# print(PROFIT)
# print("SUM: " + str(SUM))

# now = datetime.datetime.now()
# start_time = datetime.datetime(now.year, now.month, now.day, now.hour+1, 0, 0)
# print((start_time-now).total_seconds()-10)
# time.sleep((start_time-now).total_seconds()-10)
def auto():
    schedule.every().day.at("07:00").do(job)

    while True:
        schedule.run_pending()
        time.sleep(5)

# import trader
# trader.setup_demo()
COUNT += 1
#main()




# auto()
#COUNT += 1
# print(RESULT)
# print('')
# print("profit_sum: ", end='')
# print(profit_sum)
# print("pay 1000: ", end=str(profit_sum*1000))

#schedule.every(60*59).seconds.do(job)

print('Ready... :', end='')
print(datetime.datetime.now())
print("Did you restart the kernel?")



# print(RESULT)

# print('')
# print("profit_sum: ", end='')
# print(profit_sum)

print("pay 1000: ", end=str(SUM*1000))


















"AUD_JPY_2020-07-20_2021-01-03_4000.csv", AUD_JPY_2020-07-20_2021-01-03_4000.csv
Train Data
 LOW/HIGH
[1342, 1474]
money: AUD_JPY
NOW Data from140hours ago
 LOW/MID/HIGH
[45, 62]
learning data
['2020-07-20 22:00'] to ['2021-01-01 06:00']

test data
['2021-01-04 16:00'] to ['2021-01-09 02:00']

Nearest Neighbors
accuracy_score: 0.6320754716981132
confusion_matrix: 
[[28 17]
 [22 39]]
precision_score: [0.56       0.69642857]
106hours trade
profit: 21.300000000000004

CLEAR: AUD_JPY_Nearest Neighbors_4000_0.6320754716981132.sav
[0.6320754716981132]
money: AUD_JPY
NOW Data from140hours ago
 LOW/HIGH
[45, 62]

test data
['2021-01-04 16:00'] to ['2021-01-09 02:00']

models\AUD_JPY_Nearest Neighbors_4000_0.6320754716981132.sav
accuracy_score: 0.6320754716981132
precision_score: [0.56       0.69642857]
[[28 17]
 [22 39]]
[ 1.  1.  1.  1. -1.  1.  1.  1.  1. -1. -1. -1. -1. -1.  1.  1.  1.  1.
  1. -1.  1. -1.  1.  1. -1. -1.  1.  1. -1.  1.  1.  1.  1.  1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. 

<ipython-input-1-742eeea0c3b8>:556: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if(values_sum==[]):


"AUD_JPY_2020-07-08_2020-12-22_4000.csv", AUD_JPY_2020-07-08_2020-12-22_4000.csv
Train Data
 LOW/HIGH
[1363, 1477]
money: AUD_JPY
NOW Data from140hours ago
 LOW/MID/HIGH
[38, 46]
learning data
['2020-07-09 06:00'] to ['2020-12-22 14:00']

test data
['2020-12-28 19:00'] to ['2021-01-01 06:00']

Nearest Neighbors
accuracy_score: 0.4819277108433735
confusion_matrix: 
[[19 18]
 [25 21]]
precision_score: [0.43181818 0.53846154]
83hours trade
profit: -7.0

CLEAR: AUD_JPY_Nearest Neighbors_4000_0.4819277108433735.sav
[0.4819277108433735]
money: AUD_JPY
NOW Data from140hours ago
 LOW/HIGH
[38, 46]

test data
['2020-12-28 19:00'] to ['2021-01-01 06:00']

models\AUD_JPY_Nearest Neighbors_4000_0.4819277108433735.sav
accuracy_score: 0.4819277108433735
precision_score: [0.43181818 0.53846154]
[[19 18]
 [25 21]]
[-1. -1.  1. -1.  1.  1. -1. -1.  1.  1.  1.  1. -1. -1.  1. -1. -1. -1.
  1.  1.  1.  1.  1. -1.  1.  1.  1. -1.  1. -1.  1.  1. -1. -1.  1. -1.
 -1. -1.  1. -1. -1.  1. -1. -1. -1.  1.  1.

<ipython-input-1-742eeea0c3b8>:556: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if(values_sum==[]):


"AUD_JPY_2020-06-27_2020-12-10_4000.csv", AUD_JPY_2020-06-27_2020-12-10_4000.csv
Train Data
 LOW/HIGH
[1350, 1490]
money: AUD_JPY
NOW Data from140hours ago
 LOW/MID/HIGH
[24, 36]
learning data
['2020-06-29 14:00'] to ['2020-12-10 22:00']

test data
['2020-12-22 23:00'] to ['2020-12-28 10:00']

Nearest Neighbors
accuracy_score: 0.559322033898305
confusion_matrix: 
[[ 9 14]
 [12 24]]
precision_score: [0.42857143 0.63157895]
59hours trade
profit: 3.6999999999999993

CLEAR: AUD_JPY_Nearest Neighbors_4000_0.559322033898305.sav
[0.559322033898305]
money: AUD_JPY
NOW Data from140hours ago
 LOW/HIGH
[24, 36]

test data
['2020-12-22 23:00'] to ['2020-12-28 10:00']

models\AUD_JPY_Nearest Neighbors_4000_0.559322033898305.sav
accuracy_score: 0.559322033898305
precision_score: [0.42857143 0.63157895]
[[ 9 14]
 [12 24]]
[ 1.  1.  1. -1.  1.  1. -1.  1.  1.  1. -1. -1. -1.  1. -1. -1.  1.  1.
 -1.  1.  1.  1. -1.  1.  1.  1. -1.  1.  1.  1. -1. -1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1. -1. -1. -1

...

<ipython-input-1-742eeea0c3b8>:556: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if(values_sum==[]):


"AUD_JPY_2020-04-08_2020-09-21_4000.csv", AUD_JPY_2020-04-08_2020-09-21_4000.csv
Train Data
 LOW/HIGH
[1334, 1505]
money: AUD_JPY
NOW Data from140hours ago
 LOW/MID/HIGH
[51, 33]
learning data
['2020-04-08 14:00'] to ['2020-09-21 21:00']

test data
['2020-11-12 23:00'] to ['2020-11-18 10:00']

Nearest Neighbors
accuracy_score: 0.4578313253012048
confusion_matrix: 
[[22 28]
 [17 16]]
precision_score: [0.56410256 0.36363636]
83hours trade
profit: -10.799999999999997

CLEAR: AUD_JPY_Nearest Neighbors_4000_0.4578313253012048.sav
[0.4578313253012048]
money: NZD_USD
NOW Data from140hours ago
 LOW/HIGH
[44, 40]

test data
['2020-11-12 23:00'] to ['2020-11-18 10:00']

models\AUD_JPY_Nearest Neighbors_4000_0.4578313253012048.sav
accuracy_score: 0.4939759036144578
precision_score: [0.51612903 0.48076923]
[[16 27]
 [15 25]]
[-1.  1. -1. -1. -1. -1. -1.  1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1.
 -1. -1. -1.  1. -1. -1. -1.  1. -1.  1.  1.  1.  1.  1.  1.  1. -1. -1.
  1.  1.  1.  1.  1.  1. -1. 

<ipython-input-1-742eeea0c3b8>:556: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if(values_sum==[]):


"AUD_JPY_2020-03-27_2020-09-10_4000.csv", AUD_JPY_2020-03-27_2020-09-10_4000.csv
Train Data
 LOW/HIGH
[1338, 1501]
money: AUD_JPY
NOW Data from140hours ago
 LOW/MID/HIGH
[42, 42]
learning data
['2020-03-27 22:00'] to ['2020-09-10 05:00']

test data
['2020-11-07 03:00'] to ['2020-11-12 14:00']

Nearest Neighbors
accuracy_score: 0.5060240963855421
confusion_matrix: 
[[19 22]
 [19 23]]
precision_score: [0.5        0.51111111]
83hours trade
profit: -3.1999999999999957

CLEAR: AUD_JPY_Nearest Neighbors_4000_0.5060240963855421.sav
[0.5060240963855421]
money: NZD_USD
NOW Data from140hours ago
 LOW/HIGH
[44, 40]

test data
['2020-11-07 03:00'] to ['2020-11-12 14:00']

models\AUD_JPY_Nearest Neighbors_4000_0.5060240963855421.sav
accuracy_score: 0.42168674698795183
precision_score: [0.39130435 0.43333333]
[[ 9 34]
 [14 26]]
[ 1.  1. -1.  1.  1. -1.  1. -1.  1.  1.  1.  1. -1.  1.  1.  1. -1. -1.
  1.  1. -1.  1. -1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1.  1. -1.  1.
 -1.  1.  1.  1.  1.  1. -1.

<ipython-input-1-742eeea0c3b8>:556: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if(values_sum==[]):
